# NWM RouteLink file for developing topologic relationships

This notebook was adapted from original work by James Halgren (GitHub @jameshalgren).

This notebook demonstrates accessing the National Water Model (NWM) topological definition of the NWM channel routing simulation. Using these topological relationships, we can extract all of the catchments upstream of any given basin. In our use case, we find and select all NWM 3.0 reach IDs upstream of a CAMELS basin.

- RouteLink_CONUS.nc sourced from https://www.nco.ncep.noaa.gov/pmb/codes/nwprod/nwm.v3.0.13/parm/domain/RouteLink_CONUS.nc. 
- `camels_basins.txt` derived from `../01_link_usgs_nwm/output/camels_link.csv`.

Note: the source for RouteLink may change as the NWM gets updated by NOAA-OWP. Previous versions are not available for download.

Adapted by Quinn Lee (GitHub @quinnylee)

In [1]:
import xarray as xr
from collections import defaultdict
import json

In [2]:
routelink_ds = xr.open_dataset("../RouteLink_CONUS.nc")

In [3]:
# Subset the dataset to only the columns we want

subslice = [
    "link",
    "to",
    "gages",
]

routelink_df = routelink_ds[subslice].to_dataframe().astype({"link": int, "to": int,})

## Create a topology
With the downloaded Route_Link, we can generate the topology of the CONUS river network

In [4]:
routelink_df = routelink_df.set_index("link")
routelink_df

,to,gages,lon,lat
link,,,,
6635572,6635570,b' ',-96.540199,46.228783
6635590,6635600,b' ',-96.530647,46.213486
6635598,6635636,b' ',-96.505341,46.201508
6635622,6635620,b' ',-96.615021,46.200523
6635626,6635624,b' ',-96.637161,46.195522
...,...,...,...,...
15456832,25371895,b' ',-74.654648,44.979626
25371895,0,b' ',-74.648621,44.996113
15448486,0,b' ',-74.504646,44.994370


In [5]:
def replace_downstreams(data, downstream_col, terminal_code):
    '''If a node is above a terminal node, set the downstream id to the negative of the current node.'''
    ds0_mask = data[downstream_col] == terminal_code
    new_data = data.copy()
    new_data.loc[ds0_mask, downstream_col] = ds0_mask.index[ds0_mask]

    # Also set negative any nodes in downstream col not in data.index
    new_data.loc[~data[downstream_col].isin(data.index), downstream_col] *= -1
    return new_data

def reverse_network(N):
    '''
    This function was sourced from the NOAA-OWP t-route codebase
    https://github.com/NOAA-OWP/t-route

    Reverse network connections graph
    
    Arguments:
    ----------
    N (dict, int: [int]): downstream network connections
    
    Returns:
    --------
    rg (dict, int: [int]): upstream network connections
    
    '''
    rg = defaultdict(list)
    for src, dst in N.items():
        rg[src]
        for n in dst:
            rg[n].append(src)
    rg.default_factory = None
    return rg

def extract_connections(rows, target_col, terminal_codes=None):
    '''
    This function was sourced from the NOAA-OWP t-route codebase
    https://github.com/NOAA-OWP/t-route
    Extract connection network from dataframe.

    Arguments:
    ----------
    rows (DataFrame): Dataframe indexed by key_col.
    key_col    (str): Source of each edge
    target_col (str): Target of edge

    Returns:
    --------
    network (dict, int: [int]): {segment id: [list of downstream adjacent segment ids]}
    
    '''
    if terminal_codes is not None:
        terminal_codes = set(terminal_codes)
    else:
        terminal_codes = {0}

    network = {}
    for src, dst in rows[target_col].items():
        if src not in network:
            network[src] = []

        if dst not in terminal_codes:
            network[src].append(dst)
    return network

In [6]:
# Reorganize RouteLink file
routelink_df = routelink_df.sort_index()
routelink_df = replace_downstreams(routelink_df, "to", 0)

In [7]:
# Extract topology from RouteLink file
connections = extract_connections(routelink_df, "to")
rconn = reverse_network(connections)

In [8]:
def get_upstreams(basin, upstreams):
    '''Recursively get upstream basins.'''
    direct_upstreams = rconn[basin]
    for direct_upstream in direct_upstreams:
        if direct_upstream not in upstreams:
            upstreams.append(direct_upstream)
            get_upstreams(direct_upstream, upstreams)
    

In [9]:
# Read list of CAMELS basins from file
with open('camels_basins.txt', 'r') as f:
    camels_basins = f.read().splitlines()

camels_basins = [int(basin) for basin in camels_basins]

In [10]:
# Recursively get upstream basins for each CAMELS basin
upstream_dict = {}

for camels_basin in camels_basins:
    upstreams = []
    get_upstreams(camels_basin, upstreams)
    upstream_dict[camels_basin] = upstreams

In [11]:
with open('camels_upstream_dict.json', 'w') as f:
    json.dump(upstream_dict, f)